In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as f
from PIL import Image
import os
from sklearn import metrics
import glob
 

In [4]:
df=pd.read_csv("data_imu_birlestirilmis.csv")

In [5]:
label_encoder=LabelEncoder()

In [6]:
df["Class Name"]=label_encoder.fit_transform(df["Class Name"])

In [7]:
print(label_encoder.classes_)



['falling' 'not_falling']


In [8]:
df

,Unnamed: 0,ax,ay,az,gx,gy,gz,timestamp,Class Name
0,0,1.091892,8.707838,1.765225,-0.036396,0.063694,-0.036396,1900-01-01 17:44:42.000000,1
1,1,1.083568,8.732087,1.765318,-0.036781,0.064346,-0.036686,1900-01-01 17:44:42.020000,1
2,2,1.075247,8.756325,1.765411,-0.037166,0.064996,-0.036976,1900-01-01 17:44:42.040000,1
3,3,1.066932,8.780539,1.765503,-0.037551,0.065642,-0.037265,1900-01-01 17:44:42.060000,1
4,4,1.058625,8.804720,1.765596,-0.037936,0.066281,-0.037553,1900-01-01 17:44:42.080000,1
...,...,...,...,...,...,...,...,...,...
51645,51645,1.611913,8.047413,-3.202833,-0.106623,0.084735,-0.050371,1900-01-01 17:59:15.899794,1
51646,51646,1.626502,8.043946,-3.201882,-0.118920,0.123886,-0.071806,1900-01-01 17:59:15.919794,1
51647,51647,1.641042,8.040466,-3.200937,-0.131261,0.163164,-0.093307,1900-01-01 17:59:15.939794,1
51648,51648,1.655544,8.036974,-3.199998,-0.143635,0.202538,-0.114858,1900-01-01 17:59:15.959793,1


In [9]:
dataframe=df

In [10]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51650 entries, 0 to 51649
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  51650 non-null  int64  
 1   ax          51650 non-null  float64
 2   ay          51650 non-null  float64
 3   az          51650 non-null  float64
 4   gx          51650 non-null  float64
 5   gy          51650 non-null  float64
 6   gz          51650 non-null  float64
 7   timestamp   51650 non-null  object 
 8   Class Name  51650 non-null  int32  
dtypes: float64(6), int32(1), int64(1), object(1)
memory usage: 3.3+ MB


In [11]:
import torch
from torch.utils.data import Dataset
from scipy import stats

class CustomDataset(Dataset):
    def __init__(self, dataframe, sequence_length=25):
        self.dataframe = dataframe
        self.features = ["ax", "ay", "az", "gx", "gy", "gz"]
        self.labels = "Class Name"
        self.sequence_length = sequence_length

        self.scaler = StandardScaler()
        self.dataframe[self.features] = self.scaler.fit_transform(self.dataframe[self.features])

    def __len__(self):
        return len(self.dataframe) - self.sequence_length + 1

    def __getitem__(self, idx):
         
        features_sequence = self.dataframe.loc[idx:idx + self.sequence_length - 1, self.features].values.astype(float)
        
        
        label = self.dataframe.loc[idx + self.sequence_length - 1, self.labels]
        
         
        features_tensor = torch.tensor(features_sequence, dtype=torch.float32)
        label_tensor = torch.tensor(label, dtype=torch.long)

        return features_tensor, label_tensor

    

# standard scale önemli

In [12]:
dataset = CustomDataset(dataframe)

 
labels = dataset.dataframe['Class Name'].values[:len(dataset)]   

 
indices = list(range(len(dataset)))

 
train_indices, test_indices = train_test_split(indices, test_size=0.2, stratify=labels)
 
train_dataset = torch.utils.data.Subset(dataset, train_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

 
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [13]:
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 41300
Test dataset size: 10326


In [14]:
#torch.cuda.empty_cache()

In [15]:
train_dataset[6]

(tensor([[ 5.4693e-03,  8.3442e-01,  2.8824e-01, -7.3719e-01,  1.3078e+00,
           7.5716e-02],
         [ 6.3197e-03,  8.3662e-01,  2.8595e-01, -7.3963e-01,  1.3281e+00,
           7.5608e-02],
         [ 6.8068e-03,  8.3876e-01,  2.8387e-01, -7.4118e-01,  1.3458e+00,
           7.5021e-02],
         [ 6.9163e-03,  8.4082e-01,  2.8201e-01, -7.4180e-01,  1.3609e+00,
           7.3941e-02],
         [ 6.6338e-03,  8.4280e-01,  2.8040e-01, -7.4145e-01,  1.3732e+00,
           7.2352e-02],
         [ 5.9452e-03,  8.4469e-01,  2.7905e-01, -7.4010e-01,  1.3827e+00,
           7.0241e-02],
         [ 4.8361e-03,  8.4648e-01,  2.7797e-01, -7.3770e-01,  1.3891e+00,
           6.7593e-02],
         [ 3.2924e-03,  8.4818e-01,  2.7718e-01, -7.3423e-01,  1.3925e+00,
           6.4392e-02],
         [ 1.2996e-03,  8.4977e-01,  2.7670e-01, -7.2964e-01,  1.3928e+00,
           6.0624e-02],
         [-1.1564e-03,  8.5125e-01,  2.7654e-01, -7.2390e-01,  1.3897e+00,
           5.6275e-02],
         [

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [17]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        h_0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)   
        c_0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)   
        
        out, _ = self.lstm(x, (h_0, c_0))  # LSTM forward
        out = self.fc(out[:, -1, :])  # Son zaman adımının çıktısını alır
        out = self.sigmoid(out)  # Sigmoid ile iki sınıfa ayırır
        return out

In [18]:
input_size = 6  # ax, ay, az, gx, gy, gz (6 özellik)
hidden_size = 24 # Gizli katman boyutu
num_layers = 2  # LSTM katman sayısı
output_size = 1  # Binary classification
dropout_prob = 0.5  # Dropout oranı


In [19]:
model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout_prob).to(device)
criterion = nn.BCELoss()   
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
model.to(device)

LSTMModel(
  (lstm): LSTM(6, 24, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=24, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

for features,labels in train_loader:
        features.labels=features.to(device),labels.to(device)
        print(features[0])

In [21]:
num_epochs=30
for epoch in range(num_epochs):
    model.train()
    for features,labels in train_loader:
        features,labels=features.to(device),labels.to(device)
        optimizer.zero_grad()
        outputs =model(features)
        outputs = torch.squeeze(outputs)
        labels = labels.float()
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training complete") 

Epoch [1/30], Loss: 0.4667
Epoch [2/30], Loss: 0.2939
Epoch [3/30], Loss: 0.4661
Epoch [4/30], Loss: 0.3250
Epoch [5/30], Loss: 0.1770
Epoch [6/30], Loss: 0.4090
Epoch [7/30], Loss: 0.2672
Epoch [8/30], Loss: 0.1730
Epoch [9/30], Loss: 0.3190
Epoch [10/30], Loss: 0.2922
Epoch [11/30], Loss: 0.5379
Epoch [12/30], Loss: 0.1042
Epoch [13/30], Loss: 0.0943
Epoch [14/30], Loss: 0.0369
Epoch [15/30], Loss: 0.3274
Epoch [16/30], Loss: 0.0865
Epoch [17/30], Loss: 0.1950
Epoch [18/30], Loss: 0.0722
Epoch [19/30], Loss: 0.0540
Epoch [20/30], Loss: 0.0815
Epoch [21/30], Loss: 0.0887
Epoch [22/30], Loss: 0.0480
Epoch [23/30], Loss: 0.2207
Epoch [24/30], Loss: 0.1351
Epoch [25/30], Loss: 0.0227
Epoch [26/30], Loss: 0.1746
Epoch [27/30], Loss: 0.0556
Epoch [28/30], Loss: 0.1028
Epoch [29/30], Loss: 0.2108
Epoch [30/30], Loss: 0.0433
Training complete


In [25]:
model.eval()  # Modeli değerlendirme moduna al
test_loss = 0.0
correct_predictions = 0
total_predictions = 0

with torch.no_grad():  # Gradyan hesaplamasını kapat
    for features, labels in test_loader:
        features, labels = features.to(device), labels.to(device)
        
        outputs = model(features)
        outputs = torch.squeeze(outputs) 
        labels = labels.float()
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Tahminleri yap
        predicted = (outputs > 0.5).float()
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

# Sonuçları yazdır
average_test_loss = test_loss / len(test_loader)
accuracy = correct_predictions / total_predictions

print(f'Test Loss: {average_test_loss:.4f}, Accuracy: {accuracy:.4f}')

Test Loss: 0.1006, Accuracy: 0.9618


In [ ]:
# Modeli kaydet
torch.save(model.state_dict(), 'lstm_model_9618_22_09_2024.pth')


In [ ]:
# Modeli yeniden oluştur
model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout_prob).to(device)

# Ağırlıkları yükle
model.load_state_dict(torch.load('lstm_model.pth'))
